In [1]:
import torchvision
import torch
import torchvision.transforms as transforms
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
DATA_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier\Datasets\Augment"
CHECKPOINT_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier\Models\PyTorch ResNet50 Model\model_best_checkpoint.pth.tar"
ONNX_SAVE_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier\Models\Quantized Model\pytorch_resnet50.onnx"
NUM_CLASSES = 199
TRAIN_EPOCHS = 20

In [3]:
mean = [0.3882, 0.3525, 0.3215]
std = [0.3120, 0.2901, 0.2704]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std)),
])

data = torchvision.datasets.ImageFolder(root=DATA_PATH, transform=transform)

train_size = int(0.85 * len(data))
test_size = len(data) - train_size
train_data, test_data = torch.utils.data.random_split(data, [train_size, test_size])

In [4]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

In [5]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

In [6]:
def set_device():
    if torch.cuda.is_available(): dev = 'cuda:0'
    else: dev = 'cpu'
    
    return torch.device(dev)

def evaluate_model(model, test_loader):
    model.eval()
    
    predicted_correctly_on_epoch = 0
    total = 0
    
    device = set_device()
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            total += labels.size(0)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            predicted_correctly_on_epoch += (predicted == labels).sum().item()
    
    epoch_acc = 100.0 * (predicted_correctly_on_epoch / total)
    print(f"Testing dataset: {int(predicted_correctly_on_epoch)} out of {total} correct. Accuracy: {round(epoch_acc, 3)}")
    
    return epoch_acc

def save_checkpoint(model, epoch, optimizer, best_acc):
    state = {
        'epoch': epoch+1,
        'model': model.state_dict(),
        'best accuracy': best_acc,
        'optimizer': optimizer.state_dict()
    }
    
    torch.save(state, CHECKPOINT_PATH)

In [7]:
def train_nn(model, train_loader, test_loader, criterion, optimizer, n_epochs):
    device = set_device()
    best_acc = 0
    
    for epoch in tqdm(range(n_epochs)):
        print(f"Epoch: {epoch+1}")
        model.train()
        running_loss = 0.0
        running_correct = 0.0
        total = 0
        
        for data in train_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            total += labels.size(0)
            
            optimizer.zero_grad()
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            loss = criterion(outputs, labels)
            loss.backward()
            
            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (labels == predicted).sum().item()
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100.00 * (running_correct / total)
        
        print(f"Training dataset: {int(running_correct)} out of {total} correct. Accuracy: {round(epoch_acc, 3)}. Loss: {round(epoch_loss, 3)}")
        
        test_acc = evaluate_model(model, test_loader)
        
        if (test_acc > best_acc):
            best_acc = test_acc
            save_checkpoint(model, epoch, optimizer, best_acc)
        
    print("Finished")

In [8]:
resnet50 = models.resnet50(pretrained=False)
num_features = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_features, NUM_CLASSES)

device = set_device()
resnet50 = resnet50.to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet50.parameters(), lr=0.01, momentum=0.9, weight_decay=0.003)

In [9]:
train_nn(resnet50, train_loader, test_loader, loss_func, optimizer, n_epochs=TRAIN_EPOCHS)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

Epoch: 1
Training dataset: 662632 out of 1493921 correct. Accuracy: 44.355. Loss: 2.671
Testing dataset: 193069 out of 263634 correct. Accuracy: 73.234


  5%|███▊                                                                        | 1/20 [1:22:28<26:06:54, 4948.15s/it]

Epoch: 2
Training dataset: 1283963 out of 1493921 correct. Accuracy: 85.946. Loss: 0.715
Testing dataset: 215398 out of 263634 correct. Accuracy: 81.703


 10%|███████▌                                                                    | 2/20 [2:44:28<24:39:27, 4931.51s/it]

...

 95%|███████████████████████████████████████████████████████████████████████▎   | 19/20 [25:44:31<1:20:57, 4857.25s/it]

Testing dataset: 225900 out of 263634 correct. Accuracy: 85.687
Epoch: 20
Training dataset: 1351383 out of 1493921 correct. Accuracy: 90.459. Loss: 0.491


100%|█████████████████████████████████████████████████████████████████████████████| 20/20 [27:05:28<00:00, 4876.44s/it]

Testing dataset: 242646 out of 263634 correct. Accuracy: 92.039
Finished


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
checkpoint = torch.load(CHECKPOINT_PATH)
print(f"Best Epoch: {checkpoint['epoch']}, Best Accuracy: {round(checkpoint['best accuracy'], 3)}")

Best Epoch: 4, Best Accuracy: 92.766


In [12]:
resnet50 = models.resnet50(pretrained=False)
num_features = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_features, NUM_CLASSES)
resnet50.load_state_dict(checkpoint['model'])

dummy_input = torch.randn(1, 3, 256, 256, dtype=torch.float32)
torch.onnx.export(
    resnet50, 
    dummy_input, 
    ONNX_SAVE_PATH,
    verbose=False,
    export_params=True,
    opset_version=11,
    input_names = ['image'],
    output_names = ['pred']
)